# Prototype

Prototype 1 : CNN model that I found on Kaggle (https://www.kaggle.com/ahsanhabib98/jigsaw-rate-severity-of-toxic-comments-by-cnn ) 

#### Data processing

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import rankdata

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPool1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model



In [2]:
reddit_df = pd.read_csv('../data/reddit_dataset.csv')
reddit_df.head()

,Unnamed: 0,id,comment,score
0,0,1,Dumbass liberals. You’d let anyone fuck you o...,0.979
1,1,2,"Yeah ok, shut the fuck up bitch. It’s Bernie o...",0.958
2,2,3,"fuck you, you're just pretending to be blocked...",0.938
3,3,4,"""He's a borderline rapist and that's just what...",0.938
4,4,5,"If you stick your dick in crazy, the crazy sti...",0.938


In [3]:
len(reddit_df)

6000

In [4]:
reddit_df.columns

Index(['Unnamed: 0', 'id', 'comment', 'score'], dtype='object')

In [5]:
#drop unnamed:0 colunm 

In [6]:
reddit_df.drop(['Unnamed: 0'], axis=1, inplace=True)

Drop rows with no-longer availble (deleted or removed) comments.

In [7]:
for index, row in reddit_df.iterrows():
    if row['comment'] == '[removed]' or row['comment'] == '[deleted]' or row['comment'] == 'this comment is no longer availble':
        reddit_df.drop(index, axis=0, inplace=True)

Checking if I miss any deleted or removed comment

In [8]:
for index, row in reddit_df.iterrows():
    if row['comment'] == '[removed]' or row['comment'] == '[deleted]':
        print('you missed something')

It confirmed that all rows with no comments are removed. 

In [9]:
len(reddit_df)

5721

In total of 245 rowsa are deleted. It reduce the number of data but it is something that I cannot do anything about it. 

Now, aplit dateframe into two dataframe : Train, test, and validation data.

In [10]:
#reset indexing of datafram 
reddit_df.reset_index(drop=True, inplace=True)

### Text cleaning 

Text cleaning process involves removing unncessary words such as stopwords, and remove punctuations. 

In [11]:
import string 
from nltk.corpus import stopwords

In [12]:
import re
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

In [13]:
def clean_text(text):
#     #tokenize text 
    tokens = text.split()
#     remove punctuation 
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
#     # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
#     # filter oud stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
#     # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    
    return untokenize(tokens) 

In [14]:
# testing clean_test function ...
print('testing a function clean_text ...')
t = "I am saying, you cant be too generous as in, you shouldn't be too generous..."
c = clean_text(t)
print(c)

testing a function clean_text ...
saying cant generous shouldnt generous


itterate through each text in dataframe and clean each text

In [15]:
for index, row in reddit_df.iterrows():
    reddit_df.iloc[index, 1] = clean_text(row['comment'])


In [16]:
#shuffle rows 
reddit_df = reddit_df.sample(frac=1).reset_index(drop=True)

In [17]:
train_df = reddit_df.iloc[:4604,:]
val_df = reddit_df.iloc[4605:,:]

Train prototype model

In [18]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 10

In [19]:
# load in pre-tained word vectors
print("Loading word vectors...")
word2vec = {}
with open('./glove/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...


FileNotFoundError: [Errno 2] No such file or directory: './glove/glove.6B.100d.txt'

In [ ]:
sentences = train_df['comment']
target = train_df['score']

In [ ]:
# convert the sentences into sequences
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
# get word -> integer mapping
word2idx = tokenizer.word_index
# pad sequence so that we get N x T matrix
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) +1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros
            embedding_matrix[i] = embedding_vector

In [ ]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [ ]:
print('Building model...')
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Conv1D(128, 3, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.1)(x)
# x = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
output = Dense(1)(x)

In [ ]:
model = Model(input_, output)
model.summary()

In [ ]:
model.compile(
    loss='mse',
    optimizer='adam'
)

In [ ]:
r = model.fit(
    data,target,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
    shuffle=True,
)

validating model

In [ ]:
val_sequences = tokenizer.texts_to_sequences(val_df['comment'])
val_data = pad_sequences(val_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
result = model.evaluate(val_data, val_df['score'])
result

In [ ]:
score = model.predict(val_data).flatten()

In [ ]:
score[40:45]

In [ ]:
val_df['score'][40:45]

In [ ]:
data = {'comment':val_df['comment'], 'score':val_df['score'], 'model score':score}
result_df = pd.DataFrame(data=data)


In [ ]:
result_df

In [ ]:
count = 0
for index, row in result_df.iterrows():
    dif = abs(row['score']) - abs(row['model score'])
    if abs(dif) < 0.3:
        count = count + 1 
print(count)
print(count / len(result_df))